In [ ]:
import tensorflow as tf
print(tf.__version__)

2.9.2



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

class FireDetectionNet:
	@staticmethod

	def build(width, height, depth, classes):
	
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# CONV => RELU => POOL
		model.add(SeparableConv2D(16, (7, 7), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))

		# CONV => RELU => POOL
		model.add(SeparableConv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
  
    # (CONV => RELU) * 2 => POOL
		model.add(SeparableConv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(SeparableConv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))

		# first set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(128))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
	
		return model

In [ ]:
FIRE_PATH = '/content/fire_detection/Fire Dataset'
NON_FIRE_PATH = '/content/fire_detection/Non-Fire Dataset'

CLASSES = ["Fire", "Non-Fire"]

TRAIN_SPLIT = 0.75
TEST_SPLIT = 0.25

INIT_LR = 1e-3
BATCH_SIZE = 16
NUM_EPOCHS = 90

SAMPLE_SIZE = 50

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import sys

def load_dataset(dataset_path):

  image_paths = list(paths.list_images(dataset_path))
  image_data = []

  for image_path in image_paths:
    cur_image = cv2.imread(image_path)
    cur_image = cv2.resize(cur_image, (128, 128))
  
    image_data.append(cur_image)

  return np.array(image_data, dtype="float32")

print("Loading data...")
fire_data = load_dataset(FIRE_PATH)
non_fire_data = load_dataset(NON_FIRE_PATH)
print("Complete!")

In [ ]:
fire_labels = np.ones((fire_data.shape[0],))
non_fire_labels = np.zeros((non_fire_data.shape[0],))

In [ ]:
image_data = np.vstack([fire_data, non_fire_data])
labels = np.hstack([fire_labels, non_fire_labels])

In [ ]:
image_data /= 255
labels = to_categorical(labels, num_classes=2)
class_totals = labels.sum(axis=0)

In [ ]:
print(classWeight)
print(type(classWeight))
print(len(classWeight))

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size= TEST_SPLIT, random_state=42)

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
%matplotlib inline
plt.figure()
plt.imshow(trainX[1])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
from tensorflow import keras
aug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")